In [1]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "codellama/CodeLlama-34b-hf"
os.environ["report_to"] = "None"


bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16
                                )

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={'': 0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding='max_length')

/home/ksaff/miniconda3/envs/llm_jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model.safetensors.index.json
['/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00001-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00002-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00003-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00004-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00005-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--Cod

Loading checkpoint shards: 100%|██████████| 7/7 [00:19<00:00,  2.77s/it]


In [2]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [3]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [4]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 19660800 || all params: 17154187264 || trainable%: 0.11461225004381531


In [5]:
from datasets import load_dataset

data = load_dataset("/home/ksaff/Desktop/ttyd/fine_tuning/dataset/", 'csv')

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [6]:
import logging
import transformers

tokenizer.pad_token = tokenizer.eos_token
logging.getLogger('codecarbon').setLevel(logging.WARNING)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

[codecarbon INFO @ 22:51:11] [setup] RAM Tracking...
[codecarbon INFO @ 22:51:11] [setup] GPU Tracking...
[codecarbon INFO @ 22:51:11] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 22:51:11] [setup] CPU Tracking...
[codecarbon WARNING @ 22:51:11] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 22:51:12] We saw that you have a 13th Gen Intel(R) Core(TM) i5-13600KF but we don't know it. Please contact us.
[codecarbon INFO @ 22:51:12] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i5-13600KF
[codecarbon INFO @ 22:51:12] >>> Tracker's metadata:
[codecarbon INFO @ 22:51:12]   Platform system: Linux-6.5.0-14-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 22:51:12]   Python version: 3.10.13
[codecarbon INFO @ 22:51:12]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 22:51:12]   Available RAM : 31.196 GB
[codecarbon INFO @ 22:51:12]   CPU count: 20
[codecarbon INFO @ 22:51:12]   CPU model: 13th Gen Intel(R) Core(TM) i5-13600KF
[code

{'loss': 1.4805, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


[codecarbon INFO @ 22:51:46] Energy consumed for RAM : 0.000097 kWh. RAM Power : 11.698370933532715 W
[codecarbon INFO @ 22:51:46] Energy consumed for all GPUs : 0.002944 kWh. Total GPU Power : 353.064 W
[codecarbon INFO @ 22:51:46] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:51:46] 0.003396 kWh of electricity used since the beginning.
  1%|          | 2/200 [00:33<54:49, 16.61s/it]

{'loss': 1.4462, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}


[codecarbon INFO @ 22:52:01] Energy consumed for RAM : 0.000146 kWh. RAM Power : 11.698370933532715 W
[codecarbon INFO @ 22:52:01] Energy consumed for all GPUs : 0.004413 kWh. Total GPU Power : 352.57 W
[codecarbon INFO @ 22:52:01] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:52:01] 0.005091 kWh of electricity used since the beginning.
  2%|▏         | 3/200 [00:50<54:47, 16.69s/it]

...

[codecarbon INFO @ 23:45:31] Energy consumed for RAM : 0.010575 kWh. RAM Power : 11.698370933532715 W
[codecarbon INFO @ 23:45:31] Energy consumed for all GPUs : 0.318166 kWh. Total GPU Power : 351.13800000000003 W
[codecarbon INFO @ 23:45:31] Energy consumed for all CPUs : 0.038427 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:45:31] 0.367168 kWh of electricity used since the beginning.
100%|█████████▉| 199/200 [54:19<00:16, 16.31s/it]

{'loss': 0.0573, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.51}


[codecarbon INFO @ 23:45:46] Energy consumed for RAM : 0.010624 kWh. RAM Power : 11.698370933532715 W
[codecarbon INFO @ 23:45:46] Energy consumed for all GPUs : 0.319633 kWh. Total GPU Power : 352.156 W
[codecarbon INFO @ 23:45:46] Energy consumed for all CPUs : 0.038604 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:45:46] 0.368861 kWh of electricity used since the beginning.
100%|██████████| 200/200 [54:36<00:00, 16.43s/it][codecarbon INFO @ 23:45:52] Energy consumed for RAM : 0.010643 kWh. RAM Power : 11.698370933532715 W
[codecarbon INFO @ 23:45:52] Energy consumed for all GPUs : 0.320218 kWh. Total GPU Power : 352.372 W
[codecarbon INFO @ 23:45:52] Energy consumed for all CPUs : 0.038674 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:45:52] 0.369535 kWh of electricity used since the beginning.
100%|██████████| 200/200 [54:36<00:00, 16.38s/it]

{'loss': 0.0727, 'learning_rate': 0.0, 'epoch': 1.52}
{'train_runtime': 3276.0179, 'train_samples_per_second': 0.488, 'train_steps_per_second': 0.061, 'train_loss': 0.33393447525799275, 'epoch': 1.52}


TrainOutput(global_step=200, training_loss=0.33393447525799275, metrics={'train_runtime': 3276.0179, 'train_samples_per_second': 0.488, 'train_steps_per_second': 0.061, 'train_loss': 0.33393447525799275, 'epoch': 1.52})

In [7]:
trainer.save_model("2nd_try")